## Etude des sujets des amendements PLFSS

## Préparation des données

In [1]:
#!python -m pip install --upgrade pip

In [2]:
#!pip install -q --upgrade pandas  spacy  more_itertools

In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
#!python -m spacy download fr_core_news_sm > /dev/null

In [5]:
# Importing modules
import pandas as pd
amdt = pd.read_csv('textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
#amdt = pd.read_csv('https://github.com/leximpact/donnees-extraites-assemblee/raw/main/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdt.head(5)

,texteLegislatifUid,uid,dispositif,exposeSommaire
0,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000002,À la première phrase du premier alinéa de l’ar...,L’article L 531‑2 du Code de la Sécurité Socia...
1,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000010,Le premier alinéa de l’article L. 521‑1 du cod...,"Pendant plus de cinquante ans, notre politique..."
2,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000011,Dans les douze mois à compter de la promulgati...,Alors que depuis cinquante ans le principe mêm...
3,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000012,Dans les six mois suivant la promulgation de l...,"Depuis 2012, un grand nombre de mesures ont ét..."
4,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000015,Dans les six mois suivant la promulgation de l...,"Depuis 2012, un grand nombre de mesures ont ét..."


In [6]:
len(amdt) #Nombre total d'amendements

1608

In [7]:
#On regroupe dans un même texte chaque dispositif et son exposé sommaire
amdt['texte'] = amdt['dispositif'] + amdt['exposeSommaire'] 

## Nettoyage

In [8]:
#pip install nltk

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


False

In [10]:
#Tokenisation := découpage du texte en listes de mots
from nltk.tokenize import word_tokenize
tokenized = [ word_tokenize(text) for text in amdt['texte'] ] #return_token

# Normalization

### Retrait des mots de liaison (_stopword_) et de la ponctuation

In [11]:
# #Removing punctuation AND Casing
#( Casing: Est-ce vraiment utile ? Est-ce qu'on ne va pas perdre les Acronymes de vue ?)
new_tokenized = []
for token in tokenized:
    new_tokenized.append([ word.lower() for word in token if word.isalpha()])

In [12]:
print(new_tokenized[4])

['dans', 'les', 'six', 'mois', 'suivant', 'la', 'promulgation', 'de', 'la', 'loi', 'le', 'gouvernement', 'remet', 'un', 'rapport', 'au', 'parlement', 'qui', 'évalue', 'les', 'conséquences', 'des', 'mesures', 'adoptées', 'entre', 'et', 'sur', 'la', 'politique', 'rapport', 'porte', 'notamment', 'sur', 'le', 'quotient', 'familial', 'les', 'allocations', 'familiales', 'le', 'congé', 'parental', 'la', 'prime', 'de', 'naissance', 'l', 'allocation', 'de', 'base', 'de', 'la', 'prestation', 'd', 'accueil', 'du', 'jeune', 'enfant', 'et', 'les', 'modes', 'de', 'un', 'grand', 'nombre', 'de', 'mesures', 'ont', 'été', 'adoptées', 'au', 'détriment', 'des', 'familles', 'baisse', 'répétée', 'du', 'quotient', 'modulation', 'des', 'allocations', 'congé', 'parental', 'partagé', 'limitant', 'la', 'durée', 'maximale', 'pour', 'parent', 'à', 'ans', 'au', 'lieu', 'de', 'de', 'naissance', 'versée', 'après', 'la', 'naissance', 'et', 'non', 'non', 'tenues', 'en', 'termes', 'de', 'modes', 'de', 'conditions', 'plu

In [13]:
#Stop words

#On importe les mots "inutiles" (stopwords) du langage français
from nltk.corpus import stopwords 
stop_words = stopwords.words("french")
print(stop_words[:20])

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur']


In [14]:
#On importe nos propres stopwords
SW = pd.read_csv('Added_stop_words.csv')
SW = list(SW['StopWords'])
print("NOS MOTS 'INUTILES' : \n", SW)

NOS MOTS 'INUTILES' : 
 ['alinéa', 'amendement', 'º', 'L.', 'loi', 'présent', 'rapport', '1', '2020', 'II', 'phrase', '2', 'Gouvernement', 'I.', '575', 'organismes', '3', '2019', 'secteur', 'non', 'mot', 'mesure', 'Etat', 'Objet', 'objet', 'compte', 'situation', 'ans', 'propose', '4', 'III', 'également', 'congé', 'ainsi', 'afin', 'tel', 'cet']


In [15]:
#Stop Word removal
print("AVANT : \n", tokenized[4], "\n")
final_SW = SW + stop_words
tokenized = []
for token in new_tokenized:
    tokenized.append([ word for word in token if word not in final_SW])
print("APRÈS : \n", tokenized[4], "\n")

print("MOT ENLEVÉS : \n", final_SW) # On affiche ci-dessous tous les mots que l'on a retirés

AVANT : 
 ['Dans', 'les', 'six', 'mois', 'suivant', 'la', 'promulgation', 'de', 'la', 'loi', ',', 'le', 'Gouvernement', 'remet', 'un', 'rapport', 'au', 'Parlement', 'qui', 'évalue', 'les', 'conséquences', 'des', 'mesures', 'adoptées', 'entre', '2012', 'et', '2018', 'sur', 'la', 'politique', 'familiale.Ce', 'rapport', 'porte', 'notamment', 'sur', 'le', 'quotient', 'familial', ',', 'les', 'allocations', 'familiales', ',', 'le', 'congé', 'parental', ',', 'la', 'prime', 'de', 'naissance', ',', 'l', '’', 'allocation', 'de', 'base', 'de', 'la', 'prestation', 'd', '’', 'accueil', 'du', 'jeune', 'enfant', 'et', 'les', 'modes', 'de', 'garde.Depuis', '2012', ',', 'un', 'grand', 'nombre', 'de', 'mesures', 'ont', 'été', 'adoptées', 'au', 'détriment', 'des', 'familles', ':', '-', 'Baisse', 'répétée', 'du', 'quotient', 'familial-', 'Modulation', 'des', 'allocations', 'familiales-', 'Congé', 'parental', 'partagé', 'limitant', 'la', 'durée', 'maximale', 'pour', '1', 'parent', 'à', '2', 'ans', 'au', 'l

In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


False

In [17]:
#Lemmatization: on reduit les mots à leur racine
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

amdt_clean= []
for token in tokenized:
    amdt_clean.append([lemmatizer.lemmatize(word) for word in token])
print(amdt_clean[4])

['six', 'mois', 'suivant', 'promulgation', 'gouvernement', 'remet', 'parlement', 'évalue', 'conséquences', 'mesures', 'adoptées', 'entre', 'politique', 'porte', 'notamment', 'quotient', 'familial', 'allocation', 'familiales', 'parental', 'prime', 'naissance', 'allocation', 'base', 'prestation', 'accueil', 'jeune', 'enfant', 'mode', 'grand', 'nombre', 'mesures', 'adoptées', 'détriment', 'familles', 'baisse', 'répétée', 'quotient', 'modulation', 'allocation', 'parental', 'partagé', 'limitant', 'durée', 'maximale', 'parent', 'lieu', 'naissance', 'versée', 'après', 'naissance', 'tenues', 'termes', 'mode', 'condition', 'plus', 'drastiques', 'baisse', 'montants', 'allocation', 'base', 'paje', 'complément', 'mode', 'garde', 'or', 'force', 'constater', 'nombre', 'naissances', 'france', 'métropolitaine', 'a', 'baissé', 'depuis', 'selon', 'statistiques', 'insee', 'première', 'parues', 'août', 'enfants', 'moins', 'nés', 'cours', 'dernières', 'années', 'aussi', 'toutes', 'disposition', 'adoptées',

In [18]:
df = pd.DataFrame(amdt_clean)
#print(df.head(10))
df.to_csv('amdt_clean.csv')  #Chaque ligne est un amendement

# Les mots les plus cités

In [19]:
#Counting
from collections import Counter

most = []
for token in amdt_clean:
    bow = Counter(token)
    most.append(bow.most_common(30))


df = pd.DataFrame(most)
print(df.head(10))
df.to_csv('most_common_word_per_amdt.csv')  #Chaque ligne est un amendement 


                    0                 1                    2   \
0      (naissance, 15)      (enfant, 14)           (prime, 9)   
1       (politique, 5)      (sociale, 4)       (familiale, 4)   
2      (allocation, 6)   (familiales, 5)      (modulation, 5)   
3      (allocation, 4)     (adoptées, 3)       (naissance, 3)   
4      (allocation, 4)     (adoptées, 3)       (naissance, 3)   
5          (charge, 9)      (patient, 8)            (pied, 6)   
6     (médicalisée, 4)     (produits, 4)         (article, 3)   
7  (professionnels, 6)        (santé, 5)        (libéraux, 4)   
8        (activité, 4)         (taux, 4)      (occupation, 4)   
9            (mode, 3)  (financement, 3)  (établissements, 3)   

                   3                  4                  5               6   \
0        (sociale, 6)     (politique, 6)       (article, 5)       (date, 5)   
1              (a, 4)      (principe, 4)  (universalité, 4)     (enfant, 3)   
2         (revenu, 4)  (universalité, 4)       

# TF IDF
Term frequency - inverse document frequency

In [20]:
#Mise sous forme 'corpus'
corpus = []
for amdt1 in amdt_clean:
    temp = ' '.join(amdt1)
    corpus.append(temp)
    temp = ''
    
#Vectorization - Term Frequency in Global Corpus
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
tf = vectorizer.fit_transform(corpus)

print(corpus[2:4])

['douze mois compter promulgation présente gouvernement remet parlement évaluant conséquence intégration allocation familiales revenu fiscal sou condition retour universalité totale suppression modulation prévue décembre financement sécurité sociale depuis cinquante principe politique familiale reposait universalité savoir chaque famille quelles condition ressources bénéficiaient mêmes prestations leurs enfants charge financement sécurité sociale a mi place système modulation allocation modulation a effet échelonner montant allocation familiales fonction revenus fiscaux foyer donc transformer politique familiale basée redistribution horizontale partage fait familles sans enfants vers familles enfants politique sociale système redistribution verticale où partage fait foyer revenus plus élevés vers ceux revenus plus faibles pouvoir prononcer toute connaissance cause parlementaires groupe républicains sollicitent gouvernement intégration allocation familiales revenu fiscal sou condition r

In [21]:
#Features
feature_names = vectorizer.get_feature_names()
print(feature_names)

['aah', 'aasctel', 'abaissant', 'abaissement', 'abaisser', 'abandon', 'abandonnant', 'abandonne', 'abandonner', 'abandonnée', 'abandonnées', 'abandonnés', 'abattement', 'abattements', 'aberration', 'aboli', 'abondante', 'abondement', 'abonder', 'abondé', 'abonnement', 'abonnements', 'abord', 'aborder', 'abordées', 'abouti', 'abouties', 'aboutir', 'aboutira', 'aboutissant', 'aboutissement', 'aboutissent', 'aboutit', 'abrogation', 'abroger', 'abrogé', 'abrogée', 'abrogés', 'absence', 'absentes', 'absentéisme', 'absolue', 'absolument', 'absorber', 'absorbés', 'abstinent', 'abstraction', 'absurdité', 'abus', 'abuser', 'abusifs', 'abusive', 'ac', 'académique', 'académiques', 'accablants', 'accablement', 'accent', 'accentue', 'accentuées', 'acceptabilité', 'acceptable', 'acceptant', 'acceptation', 'accepte', 'accepter', 'access', 'accessibilité', 'accessible', 'accessibles', 'accession', 'accessoire', 'accessoires', 'accident', 'accise', 'accises', 'accompagnaient', 'accompagnant', 'accompag

In [22]:
#Génération du corpus index
import re
corpus_index = amdt['uid'].tolist()
print(corpus_index[:5])
#amdt['uid'].nunique() 

['AMANR5L15PO420120B2296P0D1N000002', 'AMANR5L15PO420120B2296P0D1N000010', 'AMANR5L15PO420120B2296P0D1N000011', 'AMANR5L15PO420120B2296P0D1N000012', 'AMANR5L15PO420120B2296P0D1N000015']


In [56]:
## LAST STOP HERE: POURQUOI A-T-ON UNIQUEMENT DES NAN ?

#IDF - Inverse Document Frequencies
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize and fit TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None)
tf_idf_scores = vectorizer.fit_transform(corpus)
#print(tf_idf_scores)
# get vocabulary of terms
feature_names = vectorizer.get_feature_names()
#print(type(tf_idf_scores.todense()))
print (pd.DataFrame(tf_idf_scores.todense()) )  #ICI on a bien les bonnes valeurs
print(pd.DataFrame(tf_idf_scores.todense()[0][0]))
#print(feature_names)
# create pandas DataFrame with tf-idf scores
df_tf_idf = pd.DataFrame(data = pd.DataFrame(tf_idf_scores.todense()), index=corpus_index, columns=feature_names)
print(df_tf_idf)
#df_tf_idf.fillna(0)
df_tf_idf.replace(np.nan, 0)
#df_tf_idf.reset_index() 
#df_tf_idf.rename_axis('ID') 


      0     1     2     3     4     5     6     7     8     9     ...  9772  \
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
1603   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1604   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1605   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1606   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1607   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      9773  9774  9775      9776  9777  9778  9779 

TypeError: todense() got an unexpected keyword argument 'default_fill_value'

In [44]:
for n in range(5):#range(len(df_tf_idf)):
    #print(n)
    #print(id)
    main = pd.DataFrame()
    main['Most_frequent_words'] = feature_names
    main['Word_occurrence'] = df_tf_idf.iloc[n]  # MAIN ISSUE HERE: Series definition not taking float but convert2 nan
    print(max(df_tf_idf.iloc[n]))
    print(len(df_tf_idf.iloc[n]))
    print(type(df_tf_idf.iloc[n]))
    print(max(main['Word_occurrence']))
    print(type(main['Word_occurrence']))
    print(type(main))
    print(main.head(5))
    main.sort_values(by =['Word_occurrence'], inplace = True, ascending = False, na_position='last')
    print(main.head(5))

nan
9782
<class 'pandas.core.series.Series'>
nan
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
  Most_frequent_words  Word_occurrence
0                 aah              NaN
1             aasctel              NaN
2           abaissant              NaN
3         abaissement              NaN
4            abaisser              NaN
  Most_frequent_words  Word_occurrence
0                 aah              NaN
1             aasctel              NaN
2           abaissant              NaN
3         abaissement              NaN
4            abaisser              NaN
nan
9782
<class 'pandas.core.series.Series'>
nan
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
  Most_frequent_words  Word_occurrence
0                 aah              NaN
1             aasctel              NaN
2           abaissant              NaN
3         abaissement              NaN
4            abaisser              NaN
  Most_frequent_words  Word_occurrence
0            

In [41]:
#Organizing most common words !Il faut transposer
tuple_mots = ()
for n in range(len(df_tf_idf)):
    print(n)
    id = df_tf_idf.index[id]
    print(id)
    #Creating a DF per amendment
    main = pd.DataFrame()
    main['Mots principaux'] = df_tf_idf.columns.T
    main['Occurence'] = df_tf_idf.iloc[n]
    
    
    #main['mots'] = df_tf_idf['ID']
    
    print(max(main[id]))
    #print(main.head(5))
    #main.sort_values(main[id])

    #main = main.iloc[:25] #On ne garde que les 25 mots les plus utilisés par amendement
    #tuple_mots.append(main)

0


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices